In [ ]:
# -*- coding: utf-8 -*-
import arcpy, os, math

# ===================== PARÁMETROS =====================
in_fc   = r"D:\ruta\a\tu.gdb\poligonos"              # FC de entrada (polígonos)
out_fc  = r"D:\ruta\a\tu.gdb\poligonos_centroides"   # FC de salida (puntos)
arcpy.env.overwriteOutput = True

# WKID para cálculos:
GEOG_WKID = 4326   # p.ej. WGS84 (lat/lon para DMS)
CTM_WKID  = 9377   # p.ej. MAGNA-SIRGAS / Colombia Bogota (ajústalo a tu "CTM12")
# ======================================================

def d2dms(dd):
    """
    Convierte decimal degrees -> (grados, minutos, segundos positivos).
    dd puede ser negativo; devolvemos componentes absolutos.
    """
    sgn = -1 if dd < 0 else 1
    dd = abs(dd)
    d  = int(math.floor(dd))
    m_float = (dd - d) * 60.0
    m  = int(math.floor(m_float))
    s  = (m_float - m) * 60.0
    return (d, m, s, sgn)

def ensure_fields(fc, fields_defs):
    """
    Crea campos si no existen.
    fields_defs: lista de tuplas (name, type, length, precision, scale, alias)
    """
    existing = {f.name.lower() for f in arcpy.ListFields(fc)}
    for name, ftype, length, precision, scale, alias in fields_defs:
        if name.lower() in existing:
            continue
        if ftype == "TEXT":
            arcpy.AddField_management(fc, name, ftype, field_length=length, field_alias=alias)
        else:
            arcpy.AddField_management(fc, name, ftype, field_precision=precision, field_scale=scale, field_alias=alias)

def main(in_fc, out_fc):
    # 1) Preparar SR y crear salida
    desc = arcpy.Describe(in_fc)
    sr_in = desc.spatialReference
    out_ws, out_name = os.path.split(out_fc)
    if not arcpy.Exists(out_ws):
        raise RuntimeError("No existe el workspace: {}".format(out_ws))
    arcpy.CreateFeatureclass_management(out_ws, out_name, "POINT", spatial_reference=sr_in)

    # 2) Replicar campos de atributos (excepto de sistema)
    skip_names = set([desc.OIDFieldName, desc.shapeFieldName, "Shape_Area", "Shape_Length"])
    keep_fields = []
    for f in arcpy.ListFields(in_fc):
        if f.required or f.name in skip_names:
            continue
        if f.type in ("Geometry","OID","Raster","Blob","GlobalID"):
            continue
        keep_fields.append(f)

    # Crear campos espejo
    for f in keep_fields:
        ftype = f.type
        name, alias = f.name, f.aliasName
        precision = getattr(f, "precision", None)
        scale     = getattr(f, "scale", None)
        length    = getattr(f, "length", None)
        if ftype == "String":
            arcpy.AddField_management(out_fc, name, "TEXT", field_length=length, field_alias=alias)
        elif ftype in ("Integer","SmallInteger"):
            arcpy.AddField_management(out_fc, name, "LONG" if ftype=="Integer" else "SHORT",
                                      field_precision=precision, field_alias=alias)
        elif ftype in ("Double","Single"):
            arcpy.AddField_management(out_fc, "DOUBLE" if ftype=="Double" else "FLOAT",
                                      field_precision=precision, field_scale=scale, field_alias=alias)
        elif ftype == "Date":
            arcpy.AddField_management(out_fc, name, "DATE", field_alias=alias)
        elif ftype == "GUID":
            arcpy.AddField_management(out_fc, name, "GUID", field_alias=alias)

    # 3) Campos extra solicitados
    extra_fields = [
        # NORTE
        ("Lat Grados", "SHORT",  None, None, None, "Lat Grados"),
        ("Lat Min",    "SHORT",  None, None, None, "Lat Min"),
        ("Lat Seg",    "DOUBLE", None, 10,   3,    "Lat Seg"),
        # ESTE (tal como lo pediste; añadimos Long Seg para no perder segundos)
        ("Latitud",       "SHORT",  None, None, None, "Latitud"),     # mantiene el nombre exacto
        ("Long Grados",   "SHORT",  None, None, None, "Long Grados"),
        ("Long Min",      "SHORT",  None, None, None, "Long Min"),
        ("Long Seg",      "DOUBLE", None, 10,   3,    "Long Seg"),
        # CTM12
        ("X", "DOUBLE", None, 18, 6, "X"),
        ("Y", "DOUBLE", None, 18, 6, "Y"),
    ]
    ensure_fields(out_fc, extra_fields)

    # 4) Transferir atributos y escribir centroides
    in_field_names  = [f.name for f in keep_fields]
    out_field_names = in_field_names + ["SHAPE@XY"]

    # Cursores
    with arcpy.da.SearchCursor(in_fc, in_field_names + ["SHAPE@"]) as s_cur, \
         arcpy.da.InsertCursor(out_fc, out_field_names) as i_cur:

        for row in s_cur:
            attrs = list(row[:-1])
            poly  = row[-1]
            try:
                c = poly.trueCentroid
            except:
                c = poly.centroid
            i_cur.insertRow(attrs + [(c.X, c.Y)])

    # 5) Calcular DMS y X/Y en SR proyectado
    geog_sr = arcpy.SpatialReference(GEOG_WKID)
    ctm_sr  = arcpy.SpatialReference(CTM_WKID)

    # Preparamos campos para UpdateCursor
    update_fields = ["OID@","SHAPE@"] + [ef[0] for ef in extra_fields]
    with arcpy.da.UpdateCursor(out_fc, update_fields) as ucur:
        for oid, shp, lat_g, lat_m, lat_s, latitud, lon_g, lon_m, lon_s, x_ctm, y_ctm in ucur:
            # 5.1 reproyectar punto a geográfico y a CTM
            p_geog = shp.projectAs(geog_sr)
            lon = p_geog.firstPoint.X
            lat = p_geog.firstPoint.Y

            # 5.2 convertir a DMS (valores positivos en componentes)
            d_lat, m_lat, s_lat, _sgn_lat = d2dms(lat)
            d_lon, m_lon, s_lon, _sgn_lon = d2dms(lon)

            # 5.3 X/Y proyectados
            p_ctm = shp.projectAs(ctm_sr)
            x = p_ctm.firstPoint.X
            y = p_ctm.firstPoint.Y

            # 5.4 asignar (manteniendo los nombres que pediste)
            lat_g  = d_lat
            lat_m  = m_lat
            lat_s  = round(s_lat, 3)

            # En tu ejemplo “ESTE” incluye columnas: Latitud, Long Grados, Long Min;
            # guardamos grados de LONGITUD en "Long Grados/Min/Seg" y también
            # ponemos los grados en "Latitud" tal como solicitaste.
            latitud = d_lon
            lon_g   = d_lon
            lon_m   = m_lon
            lon_s   = round(s_lon, 3)

            x_ctm = round(x, 6)
            y_ctm = round(y, 6)

            ucur.updateRow((oid, shp, lat_g, lat_m, lat_s, latitud, lon_g, lon_m, lon_s, x_ctm, y_ctm))

    arcpy.AddMessage(u"✓ Centroides y campos NORTE/ESTE/CTM12 calculados en: {}".format(out_fc))

if __name__ == "__main__":
    main(in_fc, out_fc)
